In [3]:
import socket
import ssl
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography import x509
from cryptography.hazmat.primitives import hashes



# Create a socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the server (replace 'localhost' and 12345 with your server's hostname and port)
server_hostname = 'localhost'
server_port = 12370
client_socket.connect((server_hostname, server_port))

# Load the client private key from file
with open('client-key.pem', 'rb') as key_file:
    client_private_key = serialization.load_pem_private_key(key_file.read(), password=None)

with open('server-cert.pem', 'rb') as cert_file:
    server_cert = x509.load_pem_x509_certificate(cert_file.read())
    server_public_key = server_cert.public_key()




# Wrap the client socket in an SSL context with certificate verification disabled
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)

# Disable certificate verification (not recommended for production)
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

secure_socket = context.wrap_socket(client_socket, server_hostname=server_hostname)

# Secure communication example
data = b'SSL Certification Completed'
encrypted_data = server_public_key.encrypt(data, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
secure_socket.send(encrypted_data)


secure_socket.close()
client_socket.close()